<a href="https://colab.research.google.com/github/SeanSDarcy2001/kanoldLab/blob/main/CNNforTones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import mat73
import numpy as np
import torch
import torch.nn as nn
import torchvision
import matplotlib as plt
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from torchvision import transforms
from torchvision.datasets import ImageFolder

data = mat73.loadmat('data_unmixed_and_averaged.mat')

data = data['data_unmixed']
windows = np.zeros([330, 330, 960])
tones = np.zeros([16, 960])

k = 0
for i in range(16) :
    for j in range(2) :
        for l in range(30) :
            windows[:, :, k] = data[:, :, l, i, j]
            one_hot = np.zeros(16)
            one_hot[i] = 1
            tones[:, k] = one_hot
            k += 1

print(windows[:, :, 0].shape)

transform = transforms.Resize((150,150))


trainFraction = .8
sample = np.random.uniform(size = 960) < trainFraction
xtrain = torch.from_numpy(windows[:, :, sample])
ytrain = torch.from_numpy(tones[:, sample])
xtest = torch.from_numpy(windows[:, :, ~sample])
ytest = torch.from_numpy(tones[:, ~sample])

#print(len(xtest[1, 1, :]))

print(len(xtrain[1, 1, :]), len(xtest[1, 1, :]), len(ytrain[1, :]), len(ytest[1, :]))

training_loader = torch.utils.data.DataLoader([xtrain, ytrain], batch_size=4, shuffle=True, num_workers=2)

class Classifier(nn.Module) :
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            
            nn.Conv2d(1, 32, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(32,64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
        
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(128 ,128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(128, 256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(256,256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Flatten(0),
            nn.Linear(430336,1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512,16)
        )
    def forward(self, x):
        return self.network(x) 


trainLoss = []
valLoss = []
trainAcc = []
valAcc = []
testAcc = []

model = Classifier()
optimizer =  torch.optim.SGD(model.parameters(), lr = .0001)
loss= torch.nn.CrossEntropyLoss()

def getAcc(gt, out) :
  correct = 0
  wrong = 0
  for i, k in zip(gt, out) :
    if i == k :
      correct += 1
    else :
      wrong += 1
  accuracy = correct / (correct + wrong)
  return accuracy

for epoch in range(50):  # loop over the dataset multiple times
    trainOuts = []
    valOuts = []
    epochLossT = []
    epochLossV = []
      
    trainEpochAccs = []
    model.train()
    model = model.float()

    idx = torch.randperm(xtrain.size(2))
    xtrain = xtrain[:, :, idx]
    ytrain = ytrain[:, idx]

    for i in range(len(xtrain[1, 1, :])):
        inputs = xtrain[:, :, i]
        label = ytrain[:, i]
        inputs = torch.unsqueeze(inputs, 0)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(inputs.float())
        print(outputs)
        #print(outputs.shape)
        print(label)
        #print(label.shape)

        l = loss(outputs, torch.argmax(label))
        print(l.item())
        epochLossT.append(l.item())
        a = getAcc(label, outputs)
        trainEpochAccs.append(a)
        l.backward()
        optimizer.step()

        

    testOuts = []
    testLabels = []
    model.eval()
    for k in range(len(xtest[1, 1, :])):
        inputs = xtest[:, :, k]
        labels = ytest[:, k]
        inputs = torch.unsqueeze(inputs, 0)
        testOut = model(inputs.float())
        testOuts.append(torch.argmax(testOut))
        testLabels.append(torch.argmax(labels))

    #report stats every epoch
    print("Epoch:", epoch)
    print("Training Loss:", np.mean(epochLossT))
    trainLoss.append(np.mean(epochLossT))
    print("Validation Loss:", np.mean(epochLossV))
    valLoss.append(np.mean(epochLossV))

    trainingAcc = np.mean(trainEpochAccs)
    testingAcc = getAcc(testLabels, testOuts)
    print("Train Accuracy:", trainingAcc)
    trainAcc.append(trainingAcc)
    print("Test Accuracy:", testingAcc)
    testAcc.append(testingAcc)

print('Finished Training')

plt.figure()
plt.plot(range(50), trainLoss, color='darkorange')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss vs Epoch')

plt.figure()
plt.plot(range(50), testAcc, color='darkblue')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Test Accuracy vs Epoch')


(330, 330)
759 201 759 201
tensor([-0.2118, -2.1355,  1.4632,  0.5497,  0.7522,  2.2667, -1.2580,  0.2370,
        -0.2825,  1.0367,  0.5567, -0.4986, -1.4544, -1.5554,  0.5989,  1.2956],
       grad_fn=<AddBackward0>)
tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=torch.float64)
5.6057305335998535
tensor([  6.3566,  32.6949,   2.5327,   0.4509,  -3.2428, -11.1239,  -5.0293,
          1.4227,   5.0305,  -7.8953,  -6.9801,   4.4681,  -1.5600, -11.2223,
          0.1051,   3.8418], grad_fn=<AddBackward0>)
tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=torch.float64)
0.0
tensor([  6.2644,  32.1654,   2.4899,   0.4486,  -3.1999, -10.9511,  -4.9446,
          1.3775,   4.9445,  -7.7627,  -6.8692,   4.4002,  -1.5487, -11.0317,
          0.1168,   3.7751], grad_fn=<AddBackward0>)
tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=torch.float64)
35.36528015136719
tensor([-0.3083, -0

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


tensor([-0.1565,  0.0180,  0.0623, -0.1683, -0.0812,  0.2628,  0.0115,  0.0889,
         0.0536,  0.0944, -0.0461,  0.1954, -0.0162, -0.0488,  0.1739,  0.1417],
       grad_fn=<AddBackward0>)
tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=torch.float64)
2.8047220706939697
tensor([-0.1586,  0.0071,  0.0710, -0.1987, -0.0910,  0.2670,  0.0966,  0.1010,
         0.0450,  0.1068, -0.0657,  0.2213, -0.0166, -0.0605,  0.1726,  0.1496],
       grad_fn=<AddBackward0>)
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       dtype=torch.float64)
2.7144267559051514
tensor([-0.1690, -0.0062,  0.0544, -0.2571, -0.1018,  0.2702,  0.1042,  0.1170,
         0.0385,  0.2251, -0.0875,  0.2293, -0.0304, -0.0600,  0.1420,  0.1454],
       grad_fn=<AddBackward0>)
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       dtype=torch.float64)
2.679459810256958
tensor([-0.1477, -0.0329,  0.0515, -0.2788, -0.0709,  0.2665,  0.